In [1]:
import json
import pandas as pd
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
import keras.preprocessing.text as kpt
import keras
import re 
import numpy, time
import random
import pickle
from keras.models import load_model, model_from_json
import random

height has been deprecated.



Using TensorFlow backend.


In [2]:
def transform_data_for_tokenizer(recipe_item):
    original = recipe_item.lower()
    original = re.sub(r'\s*(\d+|[./+*-])', '', original)
    original_split = original.split(" ")
    remove_list = list(set(original).symmetric_difference(units_of_food_recipe))

    keep_list = []
    for i in original_split:
        if i not in food_size and i not in units_of_food_recipe:
            keep_list.append(i)
    
    original_split = list(filter(None, keep_list))
    new = " ".join(original_split)
    return new

In [3]:
units_of_food_recipe = ["cups", "cup", "c.", "c", 
                        "fl. oz.", "fl oz", "fluid ounce", "fluid ounces",
                        "gal", "gal.", "gallon", "gallons",
                        "oz", "oz.", "ounce", "ounces", "ouncs",
                        "pt", "pt.", "pint", "pints",
                        "lb", "lb.", "pound", "pounds",
                        "qt", "qt.", "qts", "qts.", "quart", "quarts",
                        "tbsp.", "tbsp", "T", "T.", "tablespoon", "tablespoons", "tbs.", "tbs",
                        "tsp.", "tsp", "t", "t.", "teaspoon", "teaspoons",
                        "g", "g.", "gr", "gr.", "gram", "grams", "gramme", "grammes",
                        "kg", "kg.", "kilogram", "kilograms", "kilogramme", "kilogrammes",
                        "l", "l.", "liter", "liters", "litre", "litres",
                        "mg", "mg.", "milligram", "milligrams", "milligramme", "milligrammes",
                        "ml", "ml.", "milliliter", "milliliters", "millilitre", "millilitres",
                        "pinch", "pinches", "dash", "dashes", "touch", "touches", "handful", "handfuls",
                        "stick", "sticks",
                        "cans", "can",
                        "to taste",
                        "scoop", "scoops",
                        "dollop", "dollops",
                        "sprig", "sprigs",
                       "recipe",
                       "garnish", "garnished",
                       "sprinkle", "spinkled",
                       "slices",
                       "serving", "servings",
                       "ribs", "rib", "stalk",
                       "inch", "inches", "in.",
                       "drizzle", "drizzled",
                       "to taste"]

food_size = ["small", "medium",
              "large", "about"]

food_format = ["slice", "sliced",
              "dice", "diced",
              "peel", "peeled",
              "chop", "chopped",
              "julienne", "julienned",
              "trim", "trimmed",
              "sift", "sifted",
              "freshly",
              "fine", "finely",
              "drain", "drained",
              "super thinly", "thin", "thinly",
              "grate", "grated",
              "seeded",
              "rinse", "rinsed",
              "smash", "smashed",
              "membrane", "removed",
              "cleaned",
              "mince", "minced",
              "crush", "crushed",
              "about", "around",
              '<hr>']


def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [4]:
# nlp_df = pd.read_csv("../../data/nutrient/compiled/train_nn_df.csv", index_col=0)
nlp_df = pd.read_csv("../../data/nutrient/compiled/train_nn_df_MAX.csv", index_col=0)

master_df = pd.read_csv("../../data/nutrient/compiled/nutrition_master_df.csv")

with open('../../data/recipe/recipe_all.json') as f:
    recipe_all = json.load(f)
    
with open('/root/w210_capstone/data/models/NDB_tag_unique_unique_dict.json') as f:
        NDB_tag_unique_unique_dict = json.load(f)

# loading
with open('/root/w210_capstone/data/models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
    
# load json and create model
json_file = open('/root/w210_capstone/data/models/model_simple_nn.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("/root/w210_capstone/data/models/model_simple_nn_WEIGHTS.h5")
print("Loaded model from disk")

ambiguous = pd.read_csv("/root/w210_capstone/data/recipe/ambiguous_conversion.csv")

Loaded model from disk


In [25]:
random.seed(time.time())
ids = np.unique(list(recipe_all.keys()))
random.shuffle(ids)

original_text_list = []
NDB_ID_list = []
Description_list = []

for recipe_id in ids[:5]:
    print('\n')
    print(recipe_id)
    
    for ingredient in recipe_all[recipe_id]['ingredients']:
        print(ingredient)
        ingredient_filtered = transform_data_for_tokenizer(ingredient)
        tokens = tokenizer.texts_to_matrix([ingredient_filtered])
        n = 3
        prediction = loaded_model.predict(np.array(tokens))

        index_top_3_list =  loaded_model.predict(np.array(tokens)).argsort()[0][-n:][::-1]
        NDB_top_3_list = []
        percentage_top_3_list = []
        for i in index_top_3_list:   
            for key in NDB_tag_unique_unique_dict.keys():        
                if NDB_tag_unique_unique_dict[key] == i:
                    NDB_top_3_list.append(key)
                    percentage_top_3_list.append(prediction[0][i])

        itr = 0
        while itr < 3:            
            print('\t', NDB_top_3_list[itr], round(percentage_top_3_list[itr], 4), master_df[master_df['NDB_NO'] == "\"{}\"".format(NDB_top_3_list[itr].strip('"'))]['Description'].get_values()[0])
            itr += 1
        
        print('\n')
        
        raw_input_return = input()
        
        if raw_input_return == 'exit':
            break
        
        else:
            if raw_input_return:
                if raw_input_return != 'np.nan':
                    original_text_list.append(ingredient)
                    NDB_ID_list.append(raw_input_return)
                    Description_list.append(master_df[master_df['NDB_NO'] == "\"{}\"".format(raw_input_return.strip('"'))]['Description'].get_values()[0])
            else:
                if raw_input_return != 'np.nan':
                    original_text_list.append(ingredient)
                    NDB_ID_list.append(NDB_top_3_list[0])
                    Description_list.append(master_df[master_df['NDB_NO'] == "\"{}\"".format(NDB_top_3_list[0].strip('"'))]['Description'].get_values()[0])
                

temp_df = pd.DataFrame([original_text_list, NDB_ID_list, Description_list]).transpose()
temp_df.columns = nlp_df.columns
temp_df = temp_df.dropna()
nlp_df_new = pd.concat([nlp_df, temp_df])
nlp_df_new.to_csv("../../data/nutrient/compiled/train_nn_df_MAX.csv")
nlp_df = pd.read_csv("../../data/nutrient/compiled/train_nn_df_MAX.csv", index_col=0)



RECIPE_60886
1 1/2 cups frozen lima beans
	 "11312" 0.496 peas, green, frozen, unprepared (includes foods for usda's food distribution program)
	 "09055" 0.0792 blueberries, frozen, sweetened
	 "09070" 0.058 cherries, sweet, raw


"11038"
1 1/2 cups frozen peas
	 "11312" 0.984 peas, green, frozen, unprepared (includes foods for usda's food distribution program)
	 "11300" 0.0059 peas, edible-podded, raw
	 "16056" 0.0013 chickpeas (garbanzo beans, bengal gram), mature seeds, raw



Kosher salt and freshly ground pepper
	 "02047" 0.9826 salt, table
	 "02030" 0.0119 spices, pepper, black
	 "19296" 0.0008 honey


np.nan
2 tablespoons unsalted butter
	 "01145" 0.9989 butter, without salt
	 "01001" 0.0006 butter, salted
	 "12151" 0.0001 nuts, pistachio nuts, raw



1 tablespoon chopped fresh parsley
	 "11297" 0.9992 parsley, fresh
	 "02029" 0.0003 spices, parsley, dried
	 "09078" 0.0001 cranberries, raw



4 6-ounce center-cut salmon fillets (preferably wild)
	 "15238" 0.9973 fish, salmon, 

"11211"
1 pound (31-40) shrimp deveined shells and tails removed and all other black veins removed
	 "15270" 0.9636 crustaceans, shrimp, raw
	 "15143" 0.0185 crustaceans, crab, dungeness, raw
	 "20051" 0.0021 rice, white, medium-grain, enriched, cooked



8 egg whites
	 "01124" 0.9622 egg, white, raw, fresh
	 "01123" 0.0334 egg, whole, raw, fresh
	 "01125" 0.0025 egg, yolk, raw, fresh



3 scallions white and tender green parts only sliced
	 "11291" 0.8509 onions, spring or scallions (includes tops and bulb), raw
	 "11246" 0.1424 leeks, (bulb and lower leaf-portion), raw
	 "11109" 0.0011 cabbage, raw



Salt and freshly ground black pepper
	 "02030" 0.9942 spices, pepper, black
	 "20041" 0.0017 rice, brown, medium-grain, cooked (includes foods for usda's food distribution program)
	 "02047" 0.0016 salt, table


np.nan


RECIPE_48496
1 whole free-range chicken divided into 2 boneless breasts 2 wings 2 legs and 2 thighs Reserve carcass for sauce
	 "05062" 0.6239 chicken, broiler or fryer

In [26]:
len(nlp_df)

5722